In [20]:
from ortools.linear_solver import pywraplp as mip
import numpy.linalg as la
import numpy as np
import networkx as nx

# [a-c, a-d]
# [b-c, b-d]
W1 = np.array([
    [1, 0],
    [0, 1]
])

# [c-e, c-f, c-g]
W2 = np.array([
    [1, 0, 0],
    [0, 1, 1]
])

D1 = np.array([
    [0, 300],
    [300, 0]
])
D2 = np.array([
    [0, 200],
    [200, 0]
])
D3 = np.array([
    [0,   300, 330],
    [300,   0,  30],
    [330,  30,   0]
])

Nu = {}
Lambda = {}
Iota = {}
Sum = []

solver = mip.Solver('t', mip.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# ======= NU =======
for idx in range(7):
    Nu[idx] = solver.BoolVar('Nu[%i]' % idx)
    
# ======= LAMBDA =======
# limbs 1->2
srt1 = 0
srt2 = 2
for i in range(2):
    for j in range(2):
        idx1 = srt1 + i
        idx2 = srt2 + j
        Lambda[idx1, idx2] = solver.BoolVar('Lambda[%i,%i]' % (idx1, idx2))
s = solver.Sum(
    Lambda[srt1 + i, srt2 + j] * W1[i, j] for i in range(2) for j in range(2)
)
Sum.append(s)

# limbs 2->3
srt1 = 2
srt2 = 4
for i in range(2):
    for j in range(3):
        idx1 = srt1 + i
        idx2 = srt2 + j
        Lambda[idx1, idx2] = solver.BoolVar('Lambda[%i,%i]' % (idx1, idx2))
s = solver.Sum(
    Lambda[srt1 + i, srt2 + j] * W2[i, j] for i in range(2) for j in range(3)
)
Sum.append(s)

# ======= IOTA =======
# limbs 1
srt1 = 0
srt2 = 2
for i in range(2):
    for j in range(2):
        idx1 = srt1 + i
        idx2 = srt2 + j
        Iota[idx1, idx2] = solver.BoolVar('Iota[%i,%i]' % (idx1, idx2))
s = solver.Sum(
    Iota[srt1 + i, srt2 + j] * D1[i, j] for i in range(2) for j in range(2)
)
Sum.append(s)


# ====== CONSTRAINTS =======



# ==========================================
solver.Maximize(solver.Sum(Sum))
RESULT = solver.Solve()
print("Time = ", solver.WallTime(), " ms")
print("result:", RESULT)
print('\nTotal cost:', solver.Objective().Value())

for idx in range(7):
    print('node ' + str(idx) + ' -> ', Nu[idx].solution_value())

print('\n===========================')
for (a,b), v in Lambda.items():
    print(str((a, b)) + ' -> ', v.solution_value())

print('\n\n------------------')
for pid, v in Iota.items():
    print('pid:' + str(pid) + '->', v.solution_value())

Time =  2  ms
result: 0

Total cost: 605.0
node 0 ->  0.0
node 1 ->  0.0
node 2 ->  0.0
node 3 ->  0.0
node 4 ->  0.0
node 5 ->  0.0
node 6 ->  0.0

(0, 2) ->  1.0
(0, 3) ->  0.0
(1, 2) ->  0.0
(1, 3) ->  1.0
(2, 4) ->  1.0
(2, 5) ->  0.0
(2, 6) ->  0.0
(3, 4) ->  0.0
(3, 5) ->  1.0
(3, 6) ->  1.0


------------------
pid:(0, 2)-> 0.0
pid:(0, 3)-> 1.0
pid:(1, 2)-> 1.0
pid:(1, 3)-> 0.0
